* Create a function that can efficiently extract given cell_id's , the input and target from relevant h5 file.
* The output should be np.
* ie:
Lets say from metadata we take 1000 cell_id's(rows) , we need to to extract all columns related to those rows from h5 files.

In [1]:
!pip install hdf5plugin~=2.0 # https://forum.hdfgroup.org/t/cant-open-directory-usr-local-hdf5-lib-plugin/9738/4
!pip install --quiet tables
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import hdf5plugin
import tables
import psutil
import time
import scipy
import scipy.sparse as sps
import gc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 8.2 MB/s eta 0:00:0000:0100:010m


In [2]:
DATA_DIR = "/kaggle/input/open-problems-multimodal/"
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

FP_CITE_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = os.path.join(DATA_DIR,"test_cite_inputs.h5")

FP_MULTIOME_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_multi_inputs.h5")
FP_MULTIOME_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")
FP_MULTIOME_TEST_INPUTS = os.path.join(DATA_DIR,"test_multi_inputs.h5")


In [3]:
df_meta = pd.read_csv(FP_CELL_METADATA)
df_meta.head()
   

,cell_id,day,donor,cell_type,technology
0,c2150f55becb,2,27678,HSC,citeseq
1,65b7edf8a4da,2,27678,HSC,citeseq
2,c1b26cb1057b,2,27678,EryP,citeseq
3,917168fa6f83,2,27678,NeuP,citeseq
4,2b29feeca86d,2,27678,EryP,citeseq


In [4]:
df_meta.technology.unique()

array(['citeseq', 'multiome'], dtype=object)

In [5]:
f_cite_inp = h5py.File(FP_CITE_TRAIN_INPUTS,'r')#, mode)
f_cite_tgt = h5py.File(FP_CITE_TRAIN_TARGETS,'r')#, mode)  
f_multi_inp = h5py.File(FP_MULTIOME_TRAIN_INPUTS,'r')#, mode)
f_multi_tgt= h5py.File(FP_MULTIOME_TRAIN_TARGETS,'r')#, mode)

 

In [6]:
f_cite_id=f_cite_inp['train_cite_inputs']['axis1']
f_cite_ids=[str(x)[2:-1] for x in f_cite_id] 
del f_cite_id
f_multi_id=f_multi_inp['train_multi_inputs']['axis1']
f_multi_ids=[str(x)[2:-1] for x in f_multi_id] 
del f_multi_id

In [7]:
df_cite_ids=pd.DataFrame({'ids': f_cite_ids})
df_multi_ids=pd.DataFrame({'ids': f_multi_ids})

In [8]:
##checking the technology
from collections import Counter
def check_tec(cell_id):
    if type(cell_id)==str:
        tech=df_meta[df_meta["cell_id"].isin([cell_id])].technology.values
    else:
        s=set(df_meta[df_meta["cell_id"].isin(cell_id)].technology.values)
        tech = next(iter(s))
    if tech=="citeseq":
        return "cite"
    else:
        return "multi"

In [9]:
##return Array Function
def func(CELL_ID):
    tech=check_tec(CELL_ID)
    if tech=="cite":
        if type(CELL_ID)==str:
            cell_indexes=df_cite_ids[df_cite_ids.ids==CELL_ID].index.values
            if len(cell_indexes)==0:
                print("Not Found")    
             
            train_vals=f_cite_inp["train_cite_inputs"]['block0_values'][cell_indexes,:]
            target_vals=f_cite_tgt["train_cite_targets"]['block0_values'][cell_indexes,:]                          
            return cell_indexes,train_vals,target_vals
             
        else:
            cell_indexes=df_cite_ids[df_cite_ids.ids.isin(CELL_ID)].index.values
            train_vals=f_cite_inp["train_cite_inputs"]['block0_values'][cell_indexes,:]
            target_vals=f_cite_tgt["train_cite_targets"]['block0_values'][cell_indexes,:]                          
            return cell_indexes,train_vals,target_vals
    
    elif tech=='multi':
        if type(CELL_ID)==str:
            cell_indexes=df_multi_ids[df_multi_ids.ids==CELL_ID].index.values
            if len(cell_indexes)==0:
                print("Not Found")
                 
            train_vals=f_multi_inp["train_multi_inputs"]['block0_values'][cell_indexes,:]
            target_vals=f_multi_tgt["train_multi_targets"]['block0_values'][cell_indexes,:]
            return cell_indexes,train_vals,target_vals
        else:
            cell_indexes=df_multi_ids[df_multi_ids.ids.isin(CELL_ID)].index.values  
            train_vals=f_multi_inp["train_multi_inputs"]['block0_values'][cell_indexes,:]
            target_vals=f_multi_tgt["train_multi_targets"]['block0_values'][cell_indexes,:]
            return cell_indexes,train_vals,target_vals
    


In [10]:
 func('d02759a80ba2')

(array([1]),
 array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[ 8.1897014e-01,  5.0600946e-01,  1.0786819e+00,  6.8487582e+00,
          3.5248847e+00,  5.2794557e+00,  4.9304380e+00,  2.0693724e+00,
          3.3365190e-01, -4.6808821e-01,  8.6964631e-01, -3.6745235e-01,
          1.7725816e+00, -6.8803197e-01,  3.0327723e+00,  7.5592270e+00,
          4.2265887e+00,  2.8311124e+00,  5.3478870e+00, -2.9130808e-01,
          1.0382591e+00,  2.0403879e+00, -4.9186680e-01, -3.9603867e-02,
          1.7513119e+01,  6.4957762e-01, -5.3406733e-01,  4.4836575e-01,
          3.6122796e-01, -1.5066954e-01, -5.3929615e-01, -3.3843854e-01,
          4.9578232e-01,  1.5967008e+00, -5.2077621e-01, -2.9194298e-01,
          1.1366011e-01,  1.3131903e+01, -3.9918199e-01, -6.8087435e-01,
          4.9097997e-01,  4.7632280e-01, -1.2562253e-01,  4.0029892e-01,
          4.5775318e-01,  9.2730665e-01,  1.0451982e+00,  4.1284746e-01,
          7.0707793e+00,  2.2259945e-02,  4.5833895e-01

In [11]:
func(['56390cf1b95e','fc0c60183c33','15cb3d85c232'])

(array([0, 1, 4]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0.      , 0.      , 0.      , ..., 5.583255, 0.      , 4.893861],
        [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , ..., 0.      , 4.842377, 0.      ]],
       dtype=float32))